In [1]:
from path import path
import os
rootdir = path(u'/home/pfernique/Desktop/PyClangLite')
for filepath in (rootdir/'build-scons'/'src'/'wrapper').walkfiles('*.cpp'):
    if not path(filepath.replace('PyClangLite/build-scons', 'PyClangLite')).exists():
        os.remove(filepath)
        filepath = filepath.splitext()[0] + '.os'
        if filepath.exists():
            os.remove(filepath)
for filepath in (rootdir/'src'/'wrapper').walkfiles('*.[hc]*'):
    os.remove(filepath.abspath())

In [2]:
import subprocess
popen = subprocess.Popen(['clang++', '-x', 'c++', '-v', '-E', '/dev/null'], stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE)
out, err = popen.communicate()
sysincludes = err.splitlines()
headers = []
flags = ['-x', 'c++', '-std=c++11', '-D__STDC_LIMIT_MACROS', '-D__STDC_CONSTANT_MACROS']
if '#include <...> search starts here:' not in sysincludes or 'End of search list.' not in sysincludes:
    warnings.warn('System includes not computed: parsing clang command output failed', Warning)
else:
    lindex = sysincludes.index('#include <...> search starts here:')
    rindex = sysincludes.index('End of search list.')
    sysincludes = sysincludes[lindex+1:rindex]
flags.extend(['-I'+str(path(sysinclude.strip()).abspath()) for sysinclude in sysincludes])
for sysinclude in sysincludes:
    sysinclude = path(sysinclude.strip())
    clanginclude = sysinclude/'clang'
    if clanginclude.exists():
        headers.extend([sysinclude.relpathto(header) for header in clanginclude.walkfiles('*.h')])
selfcontained = rootdir/'src'/'wrapper'/'clang.h'
selfcontained.write_lines(['#include <' + str(header) + '>' for header in headers])

In [3]:
flags

['-x',
 'c++',
 '-std=c++11',
 '-D__STDC_LIMIT_MACROS',
 '-D__STDC_CONSTANT_MACROS',
 '-I/usr/include/c++/5.2.1',
 '-I/usr/include/x86_64-linux-gnu/c++/5.2.1',
 '-I/usr/include/c++/5.2.1/backward',
 '-I/usr/local/include',
 '-I/usr/local/lib/clang/3.8.0/include',
 '-I/usr/include/x86_64-linux-gnu',
 '-I/usr/include']

In [4]:
from autowig import autowig
import pickle
try:
    with open('pyclanglite.pkl', 'r') as f:
        asg = pickle.load(f)
except:
    asg = autowig.AbstractSemanticGraph()
    autowig.parser.plugin = 'pyclanglite'
    autowig.parser(asg, [selfcontained], flags=flags,
                   overload='none', bootstrap=False, permissive=True)
    with open('pyclanglite.pkl', 'w') as f:
        pickle.dump(asg, f)

In [5]:
from autowig.controller import resolve_overload, suppress_forward_declarations
resolve_overload(asg, 'none')
suppress_forward_declarations(asg)

asg['class ::std::unique_ptr'].is_copyable = False

node = asg['class ::clang::Decl']
node.is_copyable = False
for subclass in node.subclasses(recursive=True):
    subclass.is_copyable = False
    
node = asg['class ::clang::Attr']
node.is_copyable = False

node = asg['class ::clang::InheritableAttr']
node.is_copyable = False
for subclass in node.subclasses(recursive=True):
    subclass.is_copyable = False

node = asg['class ::clang::Stmt']
node.is_copyable = False
for subclass in node.subclasses(recursive=True):
    subclass.is_copyable = False
    
node = asg['class ::llvm::ArrayRef']
for specialization in node.specializations(partial=False):
    for method in specialization.methods():
        if method.localname == 'equals':
            method.boost_python_export = False

node = asg['class ::clang::TemplateArgumentListInfo']
node.is_copyable = False

node = asg['class ::clang::PreprocessedEntity']
node.is_copyable = False
for subclass in node.subclasses(recursive=True):
    subclass.is_copyable = False
    
node = asg['class ::std::initializer_list<clang::OMPClause *>']
node.is_copyable = False

node = asg['class ::llvm::SmallVectorImpl<std::pair<clang::SourceLocation, clang::PartialDiagnostic> >']
for method in node.methods(access='public'):
    if method.localname == 'resize' and len(method.parameters) == 1:
        method.boost_python_export = False
    elif method.localname.startswith('operator'):
        method.boost_python_export = False
        
#node = asg['::clang::FieldDeclarator::D']
#node.boost_python_export = False

#node = asg['class ::clang::ASTContext']
#for field in node.fields(access='public'):
#    if not getattr(field.qualified_type.desugared_type.unqualified_type, 'is_copyable', True):
#        field.boost_python_export = False
        
#node = asg['class ::clang::FileSystemStatCache']
#for method in node.methods():
#    if method.localname == 'setNextStatCache':
#        method.boost_python_export = False
        
#node = asg['class ::clang::SrcMgr::ContentCache']
#for method in node.methods():
#    if method.localname == 'setBuffer':
#        method.boost_python_export = False
        
#node = asg['class ::clang::ast_type_traits::ASTNodeKind']
#for method in node.methods():
#    if method.localname == 'getNode':
#        method.boost_python_export = False        

In [6]:
[prm.boost_python_export for prm in method.parameters]

[True]

In [7]:
%debug

ERROR: No traceback has been produced, nothing to debug.


In [8]:
function = asg.nodes('.*clang::tooling::buildASTFromCodeWithArgs').pop()
classes = [asg['class ::clang::Type'], asg['class ::clang::Decl']]
import itertools
classes.extend(itertools.chain(*[cls.subclasses(recursive=True) for cls in classes]))

In [9]:
from autowig.boost_python_generator import caching
caching(asg)
autowig.generator.plugin = 'boost_python'
wrappers = autowig.generator(asg, [function]+classes, module=rootdir/'src'/'wrapper'/'_pyclanglite.cpp',
                  decorator=None, closure=True)
if autowig.parser == 'libclang':
    pass

In [10]:
for wrapper in wrappers:
    wrapper.write()
raise

TypeError: exceptions must be old-style classes or derived from BaseException, not NoneType

In [ ]:
node = asg['class ::llvm::Optional<clang::NullabilityKind>']
for method in node.methods(access='public'):
    if method.localname.startswith('get'):
        break
        method.boost_python_export = False
    elif method.localname.startswith('operator=='):
        method.boost_python_export = False

In [ ]:
autowig.boost_python_call_policy(method) in ['boost::python::return_value_policy< boost::python::return_by_value >()', 'boost::python::return_value_policy< boost::python::reference_existing_object >()']

In [ ]:
method.return_type.desugared_type.unqualified_type

In [ ]:
node = asg['class ::llvm::Optional<clang::NullabilityKind>']
for method in node.methods(access='public'):
    print method.localname

In [ ]:
get_kind = decl[5]

In [ ]:
get_kind._valid_boost_python_export

In [ ]:
get_kind.is_virtual

In [ ]:
%debug

In [ ]:
dependencies = asg.dependencies(function, *classes)

In [ ]:
fct = asg.nodes('::clang::ASTUnit::top_level_begin.*').pop()

In [ ]:
fct.return_type

In [ ]:
len(dependencies)

In [ ]:
%debug

In [ ]:
function = asg.nodes('.*clang::tooling::buildASTFromCodeWithArgs').pop()
function.boost_python_export = True
classes = [asg['class ::clang::Type'], asg['class ::clang::Decl']]
import itertools
classes.extend(itertools.chain(*[cls.inheritors(recursive=True) for cls in classes]))
for cls in classes:
    cls.boost_python_export = True
dependencies = asg.dependencies(function, *classes, access='public')

In [ ]:
for dependency in dependencies:
    dependency.boost_python_export = True
for inheritor in asg['class ::llvm::SmallVectorBase'].inheritors(recursive=True):
    inheritor.boost_python_export = False
for specialization in asg['class ::llvm::iterator_adaptor_base'].specializations():
    specialization.boost_python_export = False
for specialization in asg['class ::llvm::iterator_facade_base'].specializations():
    specialization.boost_python_export = False
for specialization in asg['class ::llvm::SmallPtrSetIterator'].specializations():
    specialization.boost_python_export = False
for function in asg.functions(pattern='.*::set([A-Z]|_).*'):
    function.boost_python_export = False
asg['class ::clang::CFGBlock::FilterOptions'].boost_python_export = False
asg['::clang::DeclaratorChunk::PointerTypeInfo::TypeQuals'].boost_python_export = False
for namespace in asg.namespaces():
    if namespace.globalname.startswith('_'):
        for declaration in namespace.declarations(nested=True):
            declaration.boost_python_export = False
for cls in asg.classes():
    if cls.boost_python_export:
        if cls.localname.startswith('_') or any(ancestor.localname.startswith('_') for ancestor in cls.ancestors):
            cls.boost_python_export = False
for cls in asg.classes(templated=False, specialized=None):
    cls.is_copyable = False

In [ ]:
autowig.generator.plugin = 'boost_python'
autowig.generator(asg, rootdir/'src'/'wrapper'/'_pyclanglite.cpp', None, pattern='.*', closure=False)
if autowig.parser == 'libclang':
    pass

In [ ]:
module = asg[rootdir/'src'/'wrapper'/'_pyclanglite.cpp']
module.write()

for export in module.exports:
    export.write()

In [ ]:
autowig.scons(rootdir, '-j3')

In [ ]:
for node in asg.nodes():
    if hasattr(node, 'comment'):
        print node._node

In [ ]:
asg['struct ::std::__is_direct_constructible_ref_cast<const clang::CharUnits &, const clang::CharUnits &>'].comment

In [ ]:
len(module.exports)

In [ ]:
raise

In [ ]:
for node in asg.nodes():
    node.boost_python_export = False

node = asg.classes('^(class |union |struct |)::llvm::StringRef$').pop()
node.boost_python_export = True
if libclang:
    asg._nodes[node.node]['is_abstract'] = False
    asg._nodes[node.node]['_is_copyable'] = True

asg.classes('^(class |union |struct |)::clang::ASTUnit$').pop().boost_python_export = True

asg.classes('^(class |union |struct |)::clang::ASTContext$').pop().boost_python_export = True

for function in asg.functions('^::clang::ASTContext::getSourceManager'):
    if function.result_type.globalname in [prefix + '::clang::SourceManager &' for prefix in ['', 'class ']]:
        function.is_overloaded = True
        function.boost_python_export = True
        break

node = asg.classes('^(class |union |struct |)::clang::FileID$').pop()
node.boost_python_export = True
if libclang:
    asg._nodes[node.node]['is_abstract'] = False
    asg._nodes[node.node]['_is_copyable'] = True
asg.functions('^::clang::FileID::isInvalid').pop().boost_python_export = True

node = asg.classes('^(class |union |struct |)::clang::SourceLocation$').pop()
node.boost_python_export = True
if libclang:
    asg._nodes[node.node]['is_abstract'] = False
    asg._nodes[node.node]['_is_copyable'] = True

asg.classes('^(class |union |struct |)::clang::SourceManager$').pop().boost_python_export = True
for function in asg.functions('^::clang::SourceManager::getFileID'):
    if function.localname == 'getFileID':
        function.boost_python_export = True
        break
asg.functions('^::clang::SourceManager::getFilename').pop().boost_python_export = True
asg.functions('^::clang::SourceManager::getIncludeLoc').pop().boost_python_export = True

for node in asg.classes('^(class | union|struct |)::clang::([a-zA-Z]*(Decl|Type)|DeclContext)$'):
    node.boost_python_export = True

asg.enums('^(enum |)::clang::AccessSpecifier').pop().boost_python_export = True

asg.classes('^(class | union|struct |)::clang::CXXBaseSpecifier').pop().boost_python_export = True
for function in asg.functions('^::clang::CXXBaseSpecifier::getAccessSpecifier', free=False):
    if function.localname == 'getAccessSpecifier':
        function.boost_python_export = True
        break
for function in asg.functions('^::clang::CXXBaseSpecifier::getType', free=False):
    if function.localname == 'getType':
        function.boost_python_export = True
        break

node = asg.classes('^(class | union|struct |)::clang::QualType$').pop()
node.boost_python_export = True
if libclang:
    asg._nodes[node.node]['is_abstract'] = False
    asg._nodes[node.node]['_is_copyable'] = True
asg.functions('^::clang::QualType::isLocalConstQualified', free=False).pop().boost_python_export = True
asg.functions('^::clang::QualType::isLocalVolatileQualified', free=False).pop().boost_python_export = True
asg.functions('^::clang::QualType::getTypePtrOrNull', free=False).pop().boost_python_export = True

asg.functions('^(class | union|struct |)::clang::Type::isBuiltinType', free=False).pop().boost_python_export = True
for function in asg.functions('^::clang::Type::isSpecificBuiltinType', free=False):
    if function.localname == 'isSpecificBuiltinType':
        function.boost_python_export = True
        break
asg.functions('^::clang::Type::isPointerType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::isLValueReferenceType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::isRValueReferenceType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::getPointeeType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::isStructureOrClassType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::isEnumeralType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::isUnionType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::getAsTagDecl', free=False).pop().boost_python_export = True
asg.enums('(enum |)::clang::Type::TypeClass').pop().boost_python_export = True
asg.functions('^::clang::Type::isBuiltinType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::getUnqualifiedDesugaredType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::getCanonicalTypeInternal', free=False).pop().boost_python_export = True
for function in asg.functions('^::clang::Type::getTypeClass', free=False):
    if function.localname == 'getTypeClass':
        function.boost_python_export = True
        break

asg.functions('::clang::ElaboratedType::getNamedType', free=False).pop().boost_python_export = True
asg.functions('::clang::ElaboratedType::desugar', free=False).pop().boost_python_export = True

asg.enums('^(enum |)::clang::BuiltinType::Kind$').pop().boost_python_export = True

asg.functions('^::clang::Decl::isImplicit', free=False).pop().boost_python_export = True
asg.functions('^::clang::Decl::getAccessUnsafe', free=False).pop().boost_python_export = True
asg.enums('(enum |)::clang::Decl::Kind').pop().boost_python_export = True
asg.functions('^::clang::Decl::getKind').pop().boost_python_export = True
asg.functions('^::clang::Decl::getLocation').pop().boost_python_export = True
asg.functions('^::clang::Decl::getASTContext').pop().boost_python_export = True
for function in asg.functions('^::clang::Decl::getDeclContext', free=False):
    if function.result_type.globalname in [prefix + '::clang::DeclContext *' for prefix in ['', 'class ']]:
        function.is_overloaded = True
        function.boost_python_export = True
        break
for function in asg.functions('^::clang::DeclContext::getDeclKind', free=False):
    if function.localname == 'getDeclKind':
        function.boost_python_export = True
        break
for function in asg.functions('^::clang::DeclContext::getParent', free=False):
    if function.result_type.globalname in [prefix + '::clang::DeclContext *' for prefix in ['', 'class ']]:
        function.is_overloaded = True
        function.boost_python_export = True
        break
for function in asg.functions('^::clang::DeclContext::getLexicalParent', free=False):
    if function.result_type.globalname in [prefix + '::clang::DeclContext *' for prefix in ['', 'class ']]:
        function.is_overloaded = True
        function.boost_python_export = True
        break
for function in asg.functions('^::clang::DeclContext::getLookupParent', free=False):
    if function.result_type.globalname in [prefix + '::clang::DeclContext *' for prefix in ['', 'class ']]:
        function.is_overloaded = True
        function.boost_python_export = True
        break

asg.functions('^::clang::NamespaceDecl::isInline', free=False).pop().boost_python_export = True

asg.functions('^::clang::LinkageSpecDecl::getLanguage', free=False).pop().boost_python_export = True
asg.enums('(enum |)::clang::LinkageSpecDecl::LanguageIDs').pop().boost_python_export = True

asg.functions('^::clang::ValueDecl::getType').pop().boost_python_export = True

asg.functions('^::clang::FieldDecl::isMutable').pop().boost_python_export = True

for fct in asg.functions('^::clang::FunctionDecl::isDeleted'):
    if fct.localname == 'isDeleted':
        fct.boost_python_export = True
asg.functions('^::clang::FunctionDecl::getNumParams').pop().boost_python_export = True
for function in asg.functions('^::clang::FunctionDecl::getParamDecl', free=False):
    if function.result_type.globalname in [prefix + '::clang::ParmVarDecl *' for prefix in ['', 'class ']]:
        function.is_overloaded = True
        function.boost_python_export = True
for function in asg.functions('^::clang::FunctionDecl::getReturnType', free=False):
    if function.localname == 'getReturnType':
        function.boost_python_export = True
        break

for function in asg.functions('^::clang::CXXMethodDecl::isStatic', free=False):
    if function.localname == 'isStatic':
        function.boost_python_export = True
        break
asg.functions('^::clang::CXXMethodDecl::isConst', free=False).pop().boost_python_export = True
asg.functions('^::clang::CXXMethodDecl::isVolatile', free=False).pop().boost_python_export = True
asg.functions('^::clang::CXXMethodDecl::isVirtual', free=False).pop().boost_python_export = True
asg.functions('^::clang::CXXMethodDecl::isPure', free=False).pop().boost_python_export = True

asg.functions('^::clang::TagDecl::isClass', free=False).pop().boost_python_export = True
asg.functions('^::clang::TagDecl::isStruct', free=False).pop().boost_python_export = True
asg.functions('^::clang::TagDecl::isUnion', free=False).pop().boost_python_export = True
asg.functions('^::clang::TagDecl::hasNameForLinkage', free=False).pop().boost_python_export = True
asg.functions('^::clang::TagDecl::getTypedefNameForAnonDecl', free=False).pop().boost_python_export = True
for function in asg.functions('::clang::TagDecl::isCompleteDefinition', free=False):
    if function.localname == 'isCompleteDefinition':
        function.boost_python_export = True
        break

asg.functions('^::clang::CXXRecordDecl::isAbstract', free=False).pop().boost_python_export = True
asg.functions('^::clang::CXXRecordDecl::getNumBases', free=False).pop().boost_python_export = True
asg.functions('^::clang::CXXRecordDecl::getNumVBases', free=False).pop().boost_python_export = True

for function in asg.functions('^::clang::CXXConstructorDecl::isCopyConstructor'):
    if len(function.parameters) == 0:
        function.boost_python_export = True
        function.asg._nodes[function.node]['is_const'] = True
for function in asg.functions('^::clang::CXXConstructorDecl::isMoveConstructor'):
    if len(function.parameters) == 0:
        function.boost_python_export = True
        function.asg._nodes[function.node]['is_const'] = True
asg.functions('^::clang::ClassTemplateDecl::isThisDeclarationADefinition', free=False).pop().boost_python_export = True

asg.functions('^::clang::ClassTemplateSpecializationDecl::getTemplateArgs', free=False).pop().boost_python_export = True
asg.functions('^::clang::ClassTemplateSpecializationDecl::isExplicitSpecialization', free=False).pop().boost_python_export = True
for function in asg.functions('^::clang::ClassTemplateSpecializationDecl::getSpecializedTemplate', free=False):
    if function.localname == 'getSpecializedTemplate':
        function.boost_python_export = True
        break
asg.classes('^(class |union |struct |)::clang::TemplateArgumentList$').pop().boost_python_export = True
asg.functions('^::clang::TemplateArgumentList::get', free=False).pop().boost_python_export = True
asg.functions('^::clang::TemplateArgumentList::size', free=False).pop().boost_python_export = True

asg.classes('^(class |union |struct |)::clang::TemplateArgument$').pop().boost_python_export = True
asg.enums('^(enum |)::clang::TemplateArgument::ArgKind').pop().boost_python_export = True
asg.functions('^::clang::TemplateArgument::getKind', free=False).pop().boost_python_export = True
asg.functions('^::clang::TemplateArgument::getAsType', free=False).pop().boost_python_export = True
asg.functions('^::clang::TemplateArgument::getAsDecl', free=False).pop().boost_python_export = True
asg.functions('^::clang::TemplateArgument::getIntegralType', free=False).pop().boost_python_export = True

asg.functions('^::clang::TypedefNameDecl::getUnderlyingType', free=False).pop().boost_python_export = True

for enum in asg.enums():
    if not enum.clean:
        for constant in enum.constants:
            constant.boost_python_export = True

decl = asg.classes('^(class |struct |)::clang::Decl$').pop()
decl.is_copyable = False
for inheritor in decl.inheritors(True):
    inheritor.is_copyable = False

asg['::clang'].boost_python_export = True
asg['::llvm'].boost_python_export = True

if libclang:
    asg['::llvm::StringRef'].header.is_primary = True
else:
    asg['class ::llvm::StringRef'].header.is_primary = True

if libclang:
    asg['::clang::FileID'].header.is_primary = True
else:
    asg['class ::clang::FileID'].header.is_primary = True

In [ ]:
from autowig import autowig